# Sample CloudSql Connection

# Need to set up for each instance - Add your VM's IP to the Authorized Networks
**1. Find the external IP of your JupyterLab VM**
 * Go to VM Instances
 * Find your JupyterLab VM
 * Copy the External IP address (looks like 34.91.100.45).

**2. Add the VM's IP to your Cloud SQL authorized networks**
* Go to Cloud SQL instances. https://console.cloud.google.com/compute/instances?project=adsp-34002-on02-sopho-scribe&authuser=1
* Click your instance.
* Click Connections in the left sidebar.
* Scroll to Authorized networks → Add network.
* Name: anything like jupyterlab-vm
* Network: paste the external IP you just copied (e.g., 34.91.100.45/32)
* IMPORTANT: Add /32 to allow only that single IP.
* Click Save.

It will take ~30 seconds to update.

In [ ]:
#%poetry add -q google-cloud-secret-manager==2.23.3
#%poetry add -q SQLAlchemy==2.0.40

In [ ]:
#%poetry add psycopg2-binary sqlalchemy pandas

In [ ]:
from google.cloud import secretmanager
import pandas as pd
from sqlalchemy import text
from currensee.utils.db_utils import create_pg_engine

#### Create SQLAlchemy engine

In [ ]:
db_name = "postgres"

In [ ]:
engine = create_pg_engine(
   db_name = db_name
)

#### Generate & insert sample data

In [ ]:
# Create sample data
sample_data = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'score': [88, 92, 85]
})

# Create table and insert data
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS test_scores (
            id SERIAL PRIMARY KEY,
            name VARCHAR(100),
            score INT
        )
    """))

    for _, row in sample_data.iterrows():
        conn.execute(
            text("INSERT INTO test_scores (name, score) VALUES (:name, :score)"),
            {"name": row['name'], "score": row['score']}
        )

#### Read back the data

In [ ]:
df_result = pd.read_sql("SELECT * FROM test_scores", con=engine)
print(df_result)

In [ ]:
import datetime
import pytz

print(f"Notebook last execution time: {datetime.datetime.now(pytz.timezone('US/Central')).strftime('%a, %d %B %Y %H:%M:%S')}")